# Smoker Detector AI application



Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to detect cigarate smoker using a smart AI camera. In this project, we will expolre deep learning to identify a smoker in a video stream. To do this, we first build our base model. We will use a pretrained imageNet model and replace ImageNet last layer with a logistic classifier ( binary classifier ) as our overall application architecture.

**Goal** : Design and Build a Deep Learning Model to identify a cigarret smoker in a video stream.

### Base Model

The project (base model) is broken down into multiple steps:

* Load and preprocess the videos dataset if required
* Build a logistic classifier
* Train the classifier on our video dataset
* Use the trained classifier to predict

Deep Learning

![](https://drive.google.com/open?id=1TwPtHmekcdGqQeOgF3Rb_EFB1X6QIsOe)


**TO DO**

1. What input data is required? How you can represent input data i.e videos for imageNet model to consume?

  ANS 1. Input given to ImageNet would be frames of video. We could use the same code used in face_net to achieve this.


2. What should be an ideal video resolution?

  ANS 2. An ideal video resolution should be decided on the basis of, if a human can identify if there's a smoking instance in the clip with ease, that determines the ideal video resolution and anything worse than that shall be discarded from the data to be fed to the model.

3. What is the output?

  ANS 3. Output must be a '0' for non-smoking clip or '1' for smoking clip at the end of the final classifier layer.


4. What Architecture? ( with respect to the above diagram? )

  ANS 4. We are going to try Alexnet and Resnet and compare both to choose the better model. 

5. What loss function to use for this model?

  ANS 5. Logistic loss (binary cross-entropy) is the loss function to use for a binary classification problem.


First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import numpy as np
from PIL import Image
import json, time, copy
import cv2,os, glob
import pandas as pd
import seaborn as sns
from collections import OrderedDict

import torch
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.optim import lr_scheduler

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You'll also need to make sure the input data is resized to 224x224??? pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.
 

In [2]:
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'GeForce GTX 1050 Ti'

We will first extract frames out of the given video. For the time being will add a label 1 ( True ) for each frame of the video if it is from smoking folder and 0 ( False ) otherwise. In the below function, read and label data

In [3]:
print(os.getcwd())
path = '/home/anuragpatro/SmokeDetectionAI/Videos'
data_path = '/home/anuragpatro/SmokeDetectionAI/data'
def read_frame(classname,label):
  vid = 0
  for filename in os.listdir(os.path.join(path,classname)):
    vid +=1
    print("%d "%vid,end = " ")
    count = 0
    cap = cv2.VideoCapture(os.path.join(path,classname,filename))   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
      frameId = cap.get(1) #current frame number
      ret, frame = cap.read()
      if(ret != True):
        break
      imgname ="%d%dframe%d.jpg" %(label,vid,count);count+=1
      cv2.imwrite(os.path.join(data_path,imgname), frame)
    cap.release()

/home/anuragpatro/SmokeDetectionAI


In [6]:
read_frame('Smoking_dataset',1)
read_frame('NonSmoking_dataset',0)

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32 

In [4]:
# defining transforms
data_transform = {
    'train': transforms.Compose([
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

In [5]:
data_dir = '/home/anuragpatro/SmokeDetectionAI/data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

dirs = {'train': train_dir, 
        'valid': valid_dir, 
        'test' : test_dir}

In [6]:
image_datasets = {x: datasets.ImageFolder(dirs[x],transform=data_transform[x]) for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'valid', 'test']}
class_names = image_datasets['train'].classes

In [7]:
print(dataset_sizes)
class_names

{'train': 36405, 'valid': 7801, 'test': 7801}


['non_smoke', 'smoke']

### Label mapping



# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier ( Logistic Classifier ), using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters


When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.


In [19]:
model = models.alexnet(pretrained = True)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [20]:
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(9216, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, len(class_names))),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))


In [21]:
for param in model.parameters():
    param.requires_grad = False
    


In [22]:
model.classifier = classifier
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (fc1): Linear(in_features=9216, out_features=4096, bias=True)
    (relu): ReLU()
    (fc2): Linear(i

In [23]:
if torch.cuda.is_available():
    model.cuda()

In [24]:
def train_model(model, criteria, optimizer, scheduler,    
                                      num_epochs=10, device='cuda'):
    since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1,num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

#             # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model

In [26]:
# Criteria NLLLoss which is recommended with Softmax final layer
criteria = nn.NLLLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
# Number of epochs
eps=2

### Training the model on the training dataset

In [27]:
model_ft = train_model(model, criteria, optimizer, sched, eps, 'cuda')

Epoch 1/2
----------


/home/anuragpatro/anaconda3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.1827 Acc: 0.9450
valid Loss: 0.0369 Acc: 0.9886

Epoch 2/2
----------
train Loss: 0.0839 Acc: 0.9694
valid Loss: 0.0252 Acc: 0.9919

Training complete in 37m 6s
Best val Acc: 0.991924


## Testing your network

It's good practice to test your trained network on test data, images/videos the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new videos. Run the test videos through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
def calc_accuracy(model, data, cuda=False):
    correct = 0
    total = 0
    model.eval()
    model.to(device='cuda')    
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(dataloaders[data]):
            if cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            # obtain the outputs from the model
            outputs = model.forward(inputs)
            # max provides the (maximum probability, max value)
            _, predicted = outputs.max(dim=1)
            # check the 
            if idx == 0:
                print(predicted) #the predicted class
                print(torch.exp(_)) # the predicted probability
            equals = predicted == labels.data
            correct += int(torch.sum(equals == True))
            total += len(equals)
            if idx == 0:
                print(equals)
    print("Final Test Accuracy : "+str(round(correct/total,3)))

In [ ]:
calc_accuracy(model_ft,'test', True)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. 

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [29]:
# TODO: Save the checkpoint 
state = {'arch': 'alex',
            'state_dict': model.state_dict()}
torch.save(state,'alexnetsave.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [30]:
#Loading checkpoint
model = torch.load('alexnetsave.pth')


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class. Write a function called `predict` that takes an video and a model, 

First you'll need to handle processing the input image such that it can be used in your network. 

## Video Preprocessing



In [ ]:
# TODO: Process a PIL image for use in a PyTorch model


